In [327]:
import numpy as np
import pandas as pd
from matplotlib.axes import Axes
import matplotlib.pyplot as plt
from matplotlib.table import Table
from random import choice
from typing_extensions import Self, Literal, TypedDict
from typing import final
import itertools
from threading import Thread, Lock
import time
from random import seed
from multiprocessing import Process, Manager
from multiprocessing.pool import ThreadPool

In [328]:
Score = Literal['game_count', 'win_count', 'defeat_count', 'draw_count']
GameResult = Literal['win', 'defeat', 'draw']

class PlayerId(str):
	pass

class Move(int):
	pass

class MoveList(list[Move]):
	pass

class ScoreBoard(dict[Score, float]):
	pass

def Get_Player_Score_Board(score_boards: dict[PlayerId, ScoreBoard], player_id: PlayerId):
	if player_id in score_boards:
		return score_boards[player_id]
	else:
		score_board: ScoreBoard ={
			'defeat_count': 0,
			'draw_count': 0,
			'game_count': 0,
			'win_count': 0,
		}
		score_boards.update({player_id: score_board})
		return score_board

class PlayersScoreBoards(dict[PlayerId, ScoreBoard]):
	@staticmethod
	def get_score_board(score_board: dict[PlayerId, ScoreBoard], player_id: PlayerId) -> ScoreBoard:
		if player_id in score_board:
			return score_board[player_id]
		else:
			score = ScoreBoard.get_empty()
			score_board.update({player_id: score})
			return score
		


In [340]:
class Player:
	_id: PlayerId
	_score_board: ScoreBoard
	_moves: MoveList
	_original: Self|None
	_lock: Lock
	
	def __init__(self, id: PlayerId, score_board: ScoreBoard):
		self._id = id
		self._score_board = score_board
		self._moves = None
		self._original = None
		self._lock = Lock()
	
	def get_player(self) -> Self:
		'''
		По умолчанию создаёт копию вызывая `self.__copy().`

		Подготавливает "игрока" к началу игры.

		Возвращает обработанную копию. 
		'''
		copy = self.__copy()
		if copy._moves is None:
			copy._moves = []
		return copy

	def get_moves(self) -> MoveList:
		'''
		Функция обратной связи, фиксирует что ранее предложенный ход был совершён.
		.. warning:: Обязательно вызывайте `super().apply_move()`
		'''
		if self._original is None:
			raise PermissionError('Original player can`t get moves!')
		return [move_index for move_index in self._moves]
	
	def apply_move(self, move: Move):
		'''
		Функция обратной связи, фиксирует что ранее предложенный ход был совершён.
		.. warning:: Обязательно вызывайте `super().apply_move()`
		'''
		if self._original is None:
			raise PermissionError('Original player can`t apply move!')
		self._moves.append(move)

	def move(self, other_player_moves: MoveList, allowed_moves: MoveList) -> Move:
		if type(self) == Player:
			raise NotImplementedError('I can`t move!')
		if self._original is None:
			raise PermissionError('Original player can`t make moves!')
	
	def _wait_lock(self):
		# while self._lock.locked():
		# 	time.sleep(.5)
		# self._lock.acquire()
		pass

	def _unlock(self):
		# self._lock.release()
		pass
		

	def _on_game_over(self, result: GameResult):
		'''
		Выполняет после зачисление победного результата игры.
		'''
		pass

	def win(self):
		'''
		Выполняет зачисление победного результата игры.

		"Копии" же вызывают метод "оригинала" `self._original.win()`

		.. warning:: Обязательно вызывайте `super().win()`
		'''
		if self._original is not None:
			self._original.win()
			return
		
		# self._wait_lock()
		# self._score_board['win_count'] += 1
		# self._score_board['game_count'] += 1
		# self._on_game_over('win')
		# self._unlock()

	def defeat(self):
		'''
		Выполняет зачисление проигрышного результата игры.

		"Копии" же вызывают метод "оригинала" `self._original.defeat()`

		.. warning:: Обязательно вызывайте `super().defeat()`
		'''
		if self._original is not None:
			self._original.defeat()
			return
		
		# self._wait_lock()
		# self._score_board['defeat_count'] += 1
		# self._score_board['game_count'] += 1
		# self._on_game_over('defeat')
		# self._unlock()

	def draw(self):
		'''
		Выполняет зачисление ничейного результата игры.

		"Копии" же вызывают метод "оригинала" `self._original.draw()`

		.. warning:: Обязательно вызывайте `super().draw()`
		'''
		if self._original is not None:
			self._original.draw()
			return
		
		# self._wait_lock()
		# self._score_board['draw_count'] += 1
		# self._score_board['game_count'] += 1
		# self._on_game_over('draw')
		# self._unlock()

	def _on_copy(copy):
		'''
		Срабатывает после создания копии базовым методом `__copy()`
		'''
		pass

	@final
	def __copy(self) -> Self:
		'''
		Выполняет копирование игрока, передавая "копии" ссылку на "оригинал".

		Копия позволит оригиналу "участвовать" сразу в нескольких играх без конфликтов.

		.. warning:: Запрещено переопределять, используйте `_on_copy()`!
		'''
		copy = type(self)(self._id, self._score_board)
		if self._original is not None:
			copy._original = self._original
		else:
			copy._original = self
		copy._on_copy()
		return copy

In [330]:
Figure = Literal['x', 'o', 'b']
GameEndingResult = Literal['positive', 'negative', 'neutral']

class GameEnding(list[Literal['x', 'o', 'b', 'positive', 'negative', 'neutral']]):
	pass

class GameSituation(list[Figure]):
	@staticmethod
	def get_situation(player_moves: MoveList, other_player_moves: MoveList):
		situation: GameSituation = [
			'b', 'b', 'b', 
			'b', 'b', 'b',
			'b', 'b', 'b', 
		]
		for i in player_moves:
			situation[i] = 'x'
		for i in other_player_moves:
			situation[i] = 'o'
		return situation

class SituationAnalyse(TypedDict):
	distance: int
	situation: GameSituation
	result: GameEndingResult

class DistanceAnalyse(TypedDict):
	distance: int
	
	win_count: int
	loose_count: int
	draw_count: int
	total_count: int

	win_rate: float
	loose_rate: float
	draw_rate: float

class MoveAnalyse(TypedDict):
	move: Move
	distance_analyse: dict[int, DistanceAnalyse]

	awg_win_rate: float
	awg_loose_rate: float
	awg_draw_rate: float

	max_win_rate: float
	max_loose_rate: float
	max_draw_rate: float

	min_win_rate: float
	min_loose_rate: float
	min_draw_rate: float
	
	total_win_rate: float
	total_loose_rate: float
	total_draw_rate: float

	next_win_rate: float
	next_loose_rate: float
	next_draw_rate: float

	nearest_win_dist: int
	nearest_lose_dist: int
	
def Get_Distance_Analyse(distance_analyses: dict[int, DistanceAnalyse], distance: int):
	if distance in distance_analyses:
		return distance_analyses[distance]
	
	analyse: DistanceAnalyse = {
		'distance': distance,
		'loose_count': 0,
		'total_count': 0,
		'draw_count': 0,
		'win_count': 0,
	}
	distance_analyses.update({ distance: analyse })
	return analyse

def Get_Move_Analyse(move_analyses:  dict[Move, MoveAnalyse], move: Move):
	if move in move_analyses:
		return move_analyses[move]
	
	analyse: MoveAnalyse = {
		'move': move,
		'distance_analyse': {},

		'awg_win_rate': 0,
		'awg_loose_rate': 0,
		'awg_draw_rate': 0,

		'max_win_rate': 0,
		'max_loose_rate': 0,
		'max_draw_rate': 0,

		'min_win_rate': 0,
		'min_loose_rate': 0,
		'min_draw_rate': 0,
		
		'total_win_rate': 0,
		'total_loose_rate': 0,
		'total_draw_rate': 0,
		
		'next_win_rate': 0,
		'next_loose_rate': 0,
		'next_draw_rate': 0,

		'nearest_win_dist': 9,
		'nearest_lose_dist': 9,
	}
	move_analyses.update({ move: analyse })
	return analyse

class Analytic_Player_AI_Filter(list[Literal[
	'awg_win_rate',
	'awg_loose_rate',
	'awg_draw_rate',
	'max_win_rate',
	'max_loose_rate',
	'max_draw_rate',
	'min_win_rate',
	'min_loose_rate',
	'min_draw_rate',
	'total_win_rate',
	'total_loose_rate',
	'total_draw_rate',
	'next_win_rate',
	'next_loose_rate',
	'next_draw_rate',
	'nearest_win_dist',
	'nearest_lose_dist',]]):
	pass

def Get_All_Possible_Filters():
	filters_list = [
		'awg_win_rate',
		'awg_loose_rate',
		# 'awg_draw_rate',
		# 'max_win_rate',
		# 'max_loose_rate',
		# 'max_draw_rate',
		# 'min_win_rate',
		# 'min_loose_rate',
		# 'min_draw_rate',
		'total_win_rate',
		'total_loose_rate',
		# 'total_draw_rate',
		'next_win_rate',
		'next_loose_rate',
		# 'next_draw_rate',
		# 'nearest_win_dist',
		# 'nearest_lose_dist',
	]
	return itertools.permutations(filters_list, 2)
# len([i for i in Get_All_Possible_Filters()])

In [331]:
def read_endings() -> list[GameEnding]:
	endings: list[GameEnding] = np.genfromtxt('tic-tac-toe.data', delimiter=',', dtype='U30')
	for ending in endings:
		if ending[9] == 'negative' and 'b' not in ending:
			ending[9] = 'neutral'
	return endings
def is_analyzable_ending(situation: GameSituation, ending: GameEnding):
	ending_distance = 0
	for i in range(9):
		if situation[i] != ending[i]:
			if situation[i] == 'b':
				if ending[i] == 'x':
					ending_distance += 1
			else:
				return False, 99
	return True, ending_distance

class Analytic_Player(Player):
	__endings: list[GameEnding] = read_endings()
	_filters: Analytic_Player_AI_Filter
		
	def move(self, other_player_moves: MoveList, allowed_moves: MoveList) -> Move:
		moves_analyse = Analytic_Player.__analyse_moves(
			type(self), self.get_moves(), other_player_moves, allowed_moves
		)
		moves = sorted(moves_analyse, key=lambda analyse: (
			analyse[self._filters[0]] * (-1 if 'loose' in self._filters[0] else 1),
			analyse[self._filters[1]] * (-1 if 'loose' in self._filters[1] else 1),
		), reverse=True)
		
		selected_moves = [move['move'] for move in moves if (
			move[self._filters[0]] == moves[0][self._filters[0]] and
			move[self._filters[1]] == moves[0][self._filters[1]]
		)]
		# for move, analyse in moves_analyse.items():
		# 	print(move)
		# 	for metric, analyse in analyse.items():
		# 		print('\t', metric, analyse)
		# 	print()
		return choice(selected_moves)
	
	@final
	@staticmethod
	def __analyse_moves(cls: type, player_moves: MoveList, other_player_moves: MoveList, allowed_moves: MoveList):
		'''
		Вызывать используя следующий синтаксис:
		.. code-block:: python
			player = Analytic_Player(...)
			player.analyse_moves(type(player), ...)
		'''
		
		situation = GameSituation.get_situation(player_moves, other_player_moves)
		analyzable_endings: list[SituationAnalyse] = []
		for ending in Analytic_Player.__endings:
			analyzable, ending_distance = is_analyzable_ending(situation, ending)
			if not analyzable:
				continue
			analyzable_endings.append({
				'distance': ending_distance, 
				'situation': ending[:9], 
				'result': ending[9]
			})
		analyzable_endings.sort(key=lambda ending: ending['distance'])
		moves_analyses: dict[Move, MoveAnalyse] = {}
		for move in allowed_moves:
			move_analyse = Get_Move_Analyse(moves_analyses, move)
			for ending in analyzable_endings:
				if ending['situation'][move] != 'x':
					continue
				distance_analyse = Get_Distance_Analyse(move_analyse['distance_analyse'], ending['distance'])
				distance_analyse['total_count'] += 1
				if ending['result'] == 'positive':
					distance_analyse['win_count'] += 1
				elif ending['result'] == 'negative':
					distance_analyse['loose_count'] += 1
				elif ending['result'] == 'neutral':
					distance_analyse['draw_count'] += 1
					
			if len(move_analyse['distance_analyse']) == 0:
				moves_analyses.pop(move)
				continue
			
			win_rates = []
			loose_rates = []
			draw_rates = []
			total_wins = 0
			total_looses = 0
			total_draws = 0
			total_games = 0
			
			for distance, analyse in move_analyse['distance_analyse'].items():
				analyse['win_rate'] = analyse['win_count'] / analyse['total_count'] * 100	
				analyse['loose_rate'] = analyse['loose_count'] / analyse['total_count'] * 100
				analyse['draw_rate'] = analyse['draw_count'] / analyse['total_count'] * 100
				
				if distance == 1:
					move_analyse['next_win_rate'] = analyse['win_rate']
					move_analyse['next_loose_rate'] = analyse['loose_rate']
					move_analyse['next_draw_rate'] = analyse['draw_rate']
 
				if analyse['win_count'] != 0 and move_analyse['nearest_win_dist'] > analyse['distance']:
					move_analyse['nearest_win_dist'] = analyse['distance']
				if analyse['loose_count'] != 0 and move_analyse['nearest_lose_dist'] > analyse['distance']:
					move_analyse['nearest_lose_dist'] = analyse['distance']
				
				win_rates.append(analyse['win_rate'])
				loose_rates.append(analyse['loose_rate'])
				draw_rates.append(analyse['draw_rate'])

				total_wins += analyse['win_count']
				total_looses += analyse['loose_count']
				total_draws += analyse['draw_count']
				total_games += analyse['total_count']

			move_analyse['awg_win_rate'] = sum(win_rates) / len(win_rates)
			move_analyse['awg_loose_rate'] = sum(loose_rates) / len(loose_rates)
			move_analyse['awg_draw_rate'] = sum(draw_rates) / len(draw_rates)

			move_analyse['max_win_rate'] = max(win_rates)
			move_analyse['max_loose_rate'] = max(loose_rates)
			move_analyse['max_draw_rate'] = max(draw_rates)

			move_analyse['min_win_rate'] = min(win_rates)
			move_analyse['min_loose_rate'] = min(loose_rates)
			move_analyse['min_draw_rate'] = min(draw_rates)

			move_analyse['total_win_rate'] = total_wins / total_games
			move_analyse['total_loose_rate'] = total_looses / total_games
			move_analyse['total_draw_rate'] = total_draws / total_games
			
		return [analyse for move, analyse in moves_analyses.items()]
	
	def _on_copy(copy):
		copy._filters = copy._original._filters


In [332]:
class Game:
	_player_x: Player
	_player_o: Player
	_game_map: GameSituation
	_game_over: bool
	_debug: bool
	_pause_on_x: bool
	_pause_on_o: bool
	def __init__(self, debug=False, pause_on_x=False, pause_on_o=False, pause_on_move=False):
		self._player_x = None
		self._player_o = None
		self._game_map = None
		self._game_over = True
		self._debug = debug
		if pause_on_move:
			self._pause_on_x = self._pause_on_o = True
		else:
			self._pause_on_x = pause_on_x
			self._pause_on_o = pause_on_o
	
	def begin(self, player_x: Player, player_o: Player):
		game = self._copy()
		game._player_x = player_x.get_player()
		game._player_o = player_o.get_player()
		game._game_over = False
		game._game_map = [
			'b', 'b', 'b', 
			'b', 'b', 'b',
			'b', 'b', 'b',
		]
		return game

	def play(self):
		# self.draw_game_map()
		while True:
			self.player_move(self._player_x)
			if self._game_over:
				break
			# if self._pause_on_x:
			# 	input('Введите любое значение...')
			self.player_move(self._player_o)
			if self._game_over:
				break
			# if self._pause_on_o:
			# 	input('Введите любое значение...')
				
	def _copy(self):
		game = Game(self._debug, self._pause_on_x, self._pause_on_o)
		return game
	def player_move(self, player: Player):
		figure = self.get_figure(player)
		if self._debug:
			print(f'Ходит "{figure}":')
		other_player = self.get_other_player(player)
		move_index = player.move(other_player.get_moves(), self.get_allowed_moves())
		
		if self._game_map[move_index] != 'b':
			raise 'invalid move!'
		
		self._game_map[move_index] = figure
		player.apply_move(move_index)
		# self.draw_game_map()
		self.check_game_end(figure)

	def draw_game_map(self):
		if self._debug == False:
			return
		sp = plt.subplots()
		fig = sp[0]
		ax: Axes = sp[1]
		#hide the axes
		ax.axis('off')
		ax.axis('tight')

		#create data
		data = self[::, ::]
		data = [[figure if figure != 'b' else '' for figure in row] for row in data]
		df = pd.DataFrame(data)

		#create table
		table: Table = ax.table(
			cellText=df.values, colLabels=['1', '2', '3'], 
			rowLabels=['3', '2', '1'], loc='center', cellLoc='center'
		)
		table.set_fontsize(30)
		table.scale(1, 4)

		#display table
		fig.tight_layout()
		plt.show() 
	
	def get_allowed_moves(self):
		moves = []
		for i in range(len(self._game_map)):
			if self._game_map[i] == 'b':
				moves.append(i)
		return moves

	def __getitem__(self, index):
		row, col = index
		if type(row) is not slice and type(row) is not slice:
			return self._game_map[row * 3 + col]
		
		items = []
		row_indices: list = None
		col_indices: list = None

		if type(row) is slice:
			row_indices = row.indices(3)
		else:
			row_indices = [row]
		if type(col) is slice:
			col_indices = col.indices(3)
		else:
			col_indices = [col]

		for row_index in range(*row_indices):
			row_items = []
			for col_index in range(*col_indices):
				row_items.append(self._game_map[row_index * 3 + col_index])
			items.append(row_items)
		return items
	
	def get_player(self, figure):
		if figure == 'x':
			return self._player_x
		if figure == 'o':
			return self._player_o
		raise 'invalid figure!'
	
	def get_other_player(self, player: Player):
		if player == self._player_x:
			return self._player_o
		if player == self._player_o:
			return self._player_x
		raise 'invalid player!'
		
	
	def get_figure(self, player: Player):
		if player == self._player_x:
			return 'x'
		if player == self._player_o:
			return 'o'
		raise 'invalid player!'

	def check_game_end(self, figure):
		def check_win():
			# 3 по горизонтали
			if (self[0, 0] == figure and
				self[0, 1] == figure and
				self[0, 2] == figure):
				return True
			if (self[1, 0] == figure and
				self[1, 1] == figure and
				self[1, 2] == figure):
				return True
			if (self[2, 0] == figure and
				self[2, 1] == figure and
				self[2, 2] == figure):
				return True
			
			# 3 по вертикали
			if (self[0, 0] == figure and
				self[1, 0] == figure and
				self[2, 0] == figure):
				return True
			if (self[0, 1] == figure and
				self[1, 1] == figure and
				self[2, 1] == figure):
				return True
			if (self[0, 2] == figure and
				self[1, 2] == figure and
				self[2, 2] == figure):
				return True
			
			# 3 по диагонали
			if (self[0, 0] == figure and
				self[1, 1] == figure and
				self[2, 2] == figure):
				return True
			if (self[0, 2] == figure and
				self[1, 1] == figure and
				self[2, 0] == figure):
				return True
			return False

		if check_win():
			player = self.get_player(figure)
			other_player = self.get_other_player(player)
			player.win()
			other_player.defeat()
			if self._debug:
				print(f'Game over {figure} is win!')
			self._game_over = True
			return

		# Ничья
		if 'b' not in self._game_map:
			self._player_x.draw()
			self._player_o.draw()
			if self._debug:
				print(f'Game over Draw!')
			self._game_over = True

In [333]:
seed(0)
players_score_boards: dict[Player, ScoreBoard] = {}
players: list[Analytic_Player] = []
game = Game()

for filters in Get_All_Possible_Filters():
	id: PlayerId = '|'.join(filters)
	p = Analytic_Player(id, Get_Player_Score_Board(players_score_boards, id))
	p._filters = filters
	players.append(p)


In [342]:
with ThreadPool(12) as pool:
	for round in range(1):
		for player_x, player_y in itertools.permutations(players, 2):
				lobby = game.begin(player_x, player_y)
				pool.apply(lobby.play)


In [341]:
for player_x, player_y in itertools.permutations(players, 2):
	for round in range(1):
		game.begin(player_x, player_y).play()

In [336]:
for player_x, player_y in itertools.permutations(players, 2):
	for round in range(1):
		game.begin(player_x, player_y).play()

In [337]:

for player, score_board in sorted(
		players_score_boards.items(), key=lambda item: (
			item[1]['win_count'],
			-item[1]['defeat_count']
		), reverse=True
	):
	print(player, 'w|l|d: ', score_board['win_count'], score_board['defeat_count'], score_board['draw_count'], )
	

next_win_rate|total_win_rate w|l|d:  47 57 12
next_loose_rate|awg_win_rate w|l|d:  46 39 31
next_loose_rate|total_win_rate w|l|d:  46 57 13
awg_win_rate|total_win_rate w|l|d:  45 40 31
awg_win_rate|next_win_rate w|l|d:  45 42 29
next_win_rate|awg_win_rate w|l|d:  44 40 32
total_win_rate|awg_loose_rate w|l|d:  44 58 14
total_win_rate|next_win_rate w|l|d:  44 58 14
total_win_rate|next_loose_rate w|l|d:  43 57 16
total_win_rate|awg_win_rate w|l|d:  42 58 16
awg_win_rate|total_loose_rate w|l|d:  41 40 35
awg_win_rate|awg_loose_rate w|l|d:  41 41 34
awg_loose_rate|total_win_rate w|l|d:  40 22 54
total_loose_rate|total_win_rate w|l|d:  40 23 53
awg_loose_rate|awg_win_rate w|l|d:  40 33 43
awg_win_rate|next_loose_rate w|l|d:  40 39 37
total_loose_rate|awg_win_rate w|l|d:  39 27 50
total_win_rate|total_loose_rate w|l|d:  39 56 21
total_loose_rate|next_loose_rate w|l|d:  37 29 50
awg_loose_rate|next_loose_rate w|l|d:  36 29 51
next_loose_rate|next_win_rate w|l|d:  36 55 25
next_loose_rate|awg_l